In [2]:
import pymysql
from sshtunnel import SSHTunnelForwarder
from sqlalchemy import create_engine
import pandas as pd

# 建立ssh 连接
server = SSHTunnelForwarder(ssh_address_or_host=('47.254.251.32', 22),   # 指定ssh登录的跳转机的address
                            ssh_username='root',   # 跳转机的用户
                            ssh_password='Glgjssyqyhfbqz1',   # 跳转机的密码
                            remote_bind_address=('127.0.0.1', 3306))
server.start() # 开始ssh 连接

# 连接到数据库
host = '127.0.0.1'   # 此处必须是 127.0.0.1 
port = server.local_bind_port
user = 'qf'
passwd = 'root'
db = 'qf'

myConfig = pymysql.connect(user=user, passwd=passwd, 
                           host=host,
                           port=port,
                           db=db)

cursor = myConfig.cursor()

# 读取数据必须使用engine
engine = create_engine(str(r'mysql+pymysql://%s:%s@%s:%s/%s?charset=utf8') % (user, passwd, host, port, db))

D:\zANACONDA\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
sql_read = "select * from net_income_data"
net_income_data = pd.read_sql_query(sql_read, myConfig)

In [4]:
sql_read = "select * from total_assets_data"
total_assets_data = pd.read_sql_query(sql_read, myConfig)

In [5]:
sql_read = "select * from sector_info"
sector_info = pd.read_sql_query(sql_read, myConfig)

In [6]:
#行业成长能力
df = pd.merge(sector_info,net_income_data,on='Symbol',how='right')
growth_data = df[['Symbol','GICS Sector','2020','2021','2022','2023']]
result = growth_data.groupby(growth_data['GICS Sector']).apply(lambda x: x[['2020','2021','2022','2023']].sum())

In [7]:
result

,2020,2021,2022,2023
GICS Sector,,,,
Communication Services,1.452715e+11,2.680594e+11,1.700113e+11,2.462545e+11
Consumer Discretionary,2.504597e+10,9.497147e+10,4.460155e+10,1.063863e+11
Consumer Staples,4.348720e+10,5.345219e+10,5.554804e+10,6.108528e+10
Energy,-1.028389e+11,8.347245e+10,2.133365e+11,1.454511e+11
Financials,2.368669e+11,4.535100e+11,2.303653e+11,3.606796e+11
Health Care,1.437535e+11,2.172073e+11,1.982447e+11,1.663638e+11
Industrials,2.301319e+10,9.428069e+10,1.027624e+11,1.093702e+11
Information Technology,4.811653e+10,5.804150e+10,4.711770e+10,4.365561e+10
Materials,1.758926e+10,5.593020e+10,5.489923e+10,3.165996e+10


In [8]:
growth_rates = {}
for i in range(1, len(result.columns)):
    current_year = result.columns[i]
    last_year = result.columns[i-1]
    # Calculate the growth rate between the current year and the last year
    growth_rates[current_year] = (result[current_year] - result[last_year]) / result[last_year]

# Convert the growth rates to a DataFrame for display
growth_rates_df = pd.DataFrame(growth_rates)

In [9]:
growth_rates_df=growth_rates_df.reset_index()

In [10]:
growth_rates_df=growth_rates_df.rename(columns={'GICS Sector':'GICS_Sector'})

In [11]:
growth_rates_df

,GICS_Sector,2021,2022,2023
0,Communication Services,0.845231,-0.365770,0.448460
1,Consumer Discretionary,2.791886,-0.530369,1.385261
2,Consumer Staples,0.229148,0.039210,0.099684
3,Energy,-1.811682,1.555771,-0.318208
4,Financials,0.914620,-0.492039,0.565685
5,Health Care,0.510970,-0.087302,-0.160816
6,Industrials,3.096811,0.089963,0.064301
7,Information Technology,0.206270,-0.188207,-0.073477
8,Materials,2.179793,-0.018433,-0.423308
9,Real Estate,0.545563,0.051010,-0.105526


In [12]:
growth_rates_df.to_sql('industry_growth_rates', engine)

11

In [13]:
#行业盈利能力
net_income_data1 = net_income_data.set_index(["Symbol"]).drop(['index'],axis=1)
total_assets_data1 = total_assets_data.set_index(["Symbol"]).drop(['index'],axis=1)
ROE = net_income_data1.div(total_assets_data1)

df = pd.merge(sector_info,ROE,on='Symbol',how='right')
ROE_data = df[['Symbol','GICS Sector','2020','2021','2022','2023']]
result = ROE_data.groupby(ROE_data['GICS Sector']).apply(lambda x: x[['2020','2021','2022','2023']].sum())

In [14]:
profit_rates = {}
for i in range(1, len(result.columns)):
    current_year = result.columns[i]
    last_year = result.columns[i-1]
    # Calculate the profit rate between the current year and the last year
    profit_rates [current_year] = (result[current_year] - result[last_year]) / result[last_year]

# Convert the growth rates to a DataFrame for display
profit_rates_df = pd.DataFrame(profit_rates)

In [15]:
profit_rates_df=profit_rates_df.reset_index()

In [16]:
profit_rates_df=profit_rates_df.rename(columns={'GICS Sector':'GICS_Sector'})

In [17]:
profit_rates_df

,GICS_Sector,2021,2022,2023
0,Communication Services,0.810609,-0.311467,0.329709
1,Consumer Discretionary,1.795054,-0.130152,0.305431
2,Consumer Staples,0.140479,-0.104139,0.092443
3,Energy,-1.500776,1.596937,-0.302593
4,Financials,0.300583,-0.416349,0.218805
5,Health Care,0.193378,-0.348352,-0.070685
6,Industrials,0.549211,0.073732,-0.067006
7,Information Technology,0.095149,-0.029156,-0.050512
8,Materials,1.118339,0.046705,-0.378984
9,Real Estate,0.406507,0.070523,-0.218128


In [18]:
profit_rates_df.to_sql('industry_profit_rates', engine)

11